# 초기 설정

## 크롬 드라이버 세팅

In [ ]:
# chrome_driver = '../chromedriver.exe' # 파일 확장자 이름 표기
chrome_driver = '../chromedriver' # 파일 확장자 이름 미표기

## 수집할 앱 주소(구글 플레이 스토어)

In [ ]:
url = 'https://play.google.com/store/apps/details?id=com.github.android' # 예시: Github App 주소

# 패키지 import

In [1]:
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
from time import sleep

from tqdm.auto import tqdm, trange
import pandas as pd

# 웹 크롤링

## 무한 스크롤

In [6]:
def scrolling():
    try:        
        # 스크롤 높이 받아오기
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            pause_time = random.uniform(0.5, 0.8)
            # 최하단까지 스크롤
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            # 페이지 로딩 대기
            time.sleep(pause_time)
            # 무한 스크롤 동작을 위해 살짝 위로 스크롤
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);")
            time.sleep(pause_time)
            # 스크롤 높이 새롭게 받아오기
            new_height = driver.execute_script("return document.body.scrollHeight")
            try:
                # '더보기' 버튼 있을 경우 클릭
                more_btn = driver.find_element_by_xpath('/html/body/div[1]/div[4]/c-wiz/div/div[2]/div/div/main/div/div[1]/div[2]/div[2]/div/span/span').click()
            except:
                # 스크롤 완료 경우
                if new_height == last_height:
                    print("Scrolling is completed!!!")
                    break
                last_height = new_height
                
    except Exception as e:
        print("error occurred: ", e)

## HTML 데이터 가져오기

In [9]:
driver = webdriver.Chrome(chrome_driver)
# 페이지열기
driver.get(url)
# 페이지 로딩 대기
wait = WebDriverWait(driver, 5)
xpath_code = '/html/body/div[1]/div[4]/c-wiz/div/div[2]/div/div/main/div/div[1]/div[2]/div/div[1]/div/div[2]/div[1]/div[2]/div/div[2]/span/span/span'
chk_loading = wait.until(EC.element_to_be_clickable((By.XPATH, xpath_code)))
# 페이지 무한 스크롤 다운
scrolling()
# html parsing하기
html_src = driver.page_source
soup_src = BeautifulSoup(html_src, 'html.parser')

Scrolling is completed!!!


NameError: name 'save_soup' is not defined

## HTML 데이터 저장

In [121]:
# html 데이터 저장
with open("../dataset/html_data.html", "w", encoding = 'utf-8') as file:
    file.write(str(soup_src))

## 데이터프레임 형태로 변형

In [210]:
start = time.time() # 코드 실행 시간 측정을 위한 변수
date_ymd = [] # 리뷰등록일을 yyyymmdd 형태로 저장할 리스트 생성
date_y = [] # 리뷰등록일 중 연도 정보를 yyyy 형태로 저장할 리스트 생성
date_m = [] # 리뷰등록일 중 월 정보를 mm 형태 저장할 리스트 생성
date_d = [] # 리뷰등록일 중 일 정보를 dd 형태로 저장할 리스트 생성
username_list = [] # 사용자 닉네임 저장용 리스트
rating_list = [] # 평점 데이터 저장용 리스트
content_list = [] # 텍스트 리뷰 저장용 리스트

# 리뷰 1개씩 접근해 정보 추출
for rv in tqdm(review_all):
    
    date_ymd_v = rv.find_all(class_ = 'p2TkOb')[0].text
    date_y_v = date_ymd_v[0:4] # 연도 정보만 추출
    # 해당 단어가 등장한 인덱스 추출
    idx_y = date_ymd_v.find('년')
    idx_m = date_ymd_v.find('월')
    idx_d = date_ymd_v.find('일')
    date_m_v = str(int(date_ymd_v[idx_y+1:idx_m])) # 월 정보만 추출
    date_d_v = str(int(date_ymd_v[idx_m+1:idx_d])) # 일 정보만 추출
    
    # 월 정보가 1자리의 경우 앞에 0 붙여줌(e.g., 1월 -> 01월)
    if len(date_m_v) == 1:
        date_m_v = '0' + date_m_v
    # 일 정보가 1자리의 경우 앞에 0 붙여줌(e.g., 7일 -> 07일)
    if len(date_d_v) == 1:
        date_d_v = '0' + date_d_v
    
    # 리뷰등록일 full version은 최종적으로 yyyymmdd 형태로 저장
    date_full = date_y_v + date_m_v + date_d_v
    date_ymd.append(date_full)
    date_y.append(date_y_v)
    date_m.append(date_m_v)
    date_d.append(date_d_v)
    username_list.append(rv.find_all(class_ = 'X43Kjb')[0].text) # 닉네임 정보 추출 및 저장
    rating_list.append(rv.select('span.nt2C1d > .pf5lIe > div')[0]['aria-label'][10]) # 평점 정보 추출 및 저장
    content = rv.find_all('span', attrs={'jsname':"fbQN7e"})[0].text # 장문 리뷰 내용 추출 및 저장
    # 장문 리뷰 존재하는 경우 그대로 리스트에 저장
    if content:
        content_list.append(content)
    # 단문 리뷰만 존재하는 경우, 단문 리뷰 추출 및 저장
    else:
        content_list.append(rv.find_all('span', attrs={'jsname':"bN97Pc"})[0].text)
# 코드 실행 소요시간 출력
print(time.time() - start)

  0%|          | 0/23215 [00:00<?, ?it/s]

39.170166015625


## 데이터프레임 저장

In [211]:
start = time.time() # 코드 실행 시간 측정을 위한 변수
rv_df = pd.DataFrame({'id': range(len(date_ymd)), # userID 임의부여
                      'date': date_ymd, # 리뷰등록일 전체(yyyymmdd)
                      'date_y': date_y, # 리뷰등록일 중 연도(yyyy)
                      'date_m': date_m, # 리뷰등록일 중 월(mm)
                      'date_d': date_d, # 리뷰등록일 중 일(dd)
                     'username': username_list, # 사용자 닉네임
                     'rating': rating_list, # 평점
                     'content': content_list}) # 리뷰 내용
rv_df.to_csv('../dataset/review_dataset.csv', encoding = 'utf-8-sig') # csv 파일로 저장

print(time.time() - start) # 코드 실행 소요시간 출력

0.18182086944580078


# 리뷰 데이터 불러오기

In [3]:
# 저장한 리뷰 정보 불러오기
rv_df = pd.read_csv('../dataset/review_dataset.csv', encoding = 'utf-8-sig')
rv_df = rv_df.drop(['Unnamed: 0'], axis = 1) # 불필요한 칼럼 삭제
rv_df.head()

,id,date,date_y,date_m,date_d,username,rating,content
0,0,20210901,2021,9,1,맑은강물,1,핸폰바꿔서 다시 시작하는데 휴대폰인증 단계에서 인증번호요청 그게 안됩니다. 몇번이고...
1,1,20210827,2021,8,27,김미룡시,1,업데이트 공지 보고 설정에 구글플레이스토어 캐시및데이터 삭제하고 재설치하고 마지막 ...
2,2,20210902,2021,9,2,김태환,1,가끔 어플 켜지도 않았었는데 접속하면 지문인식 5회 실패라고 뜹니다 이거 너무 불편...
3,3,20210829,2021,8,29,aAAa 1,1,아니...폰바꾸고 지문등록하고 잘들어가졌는데 지문추가해서 지문재등록하라고 뜬뒤로 안...
4,4,20210829,2021,8,29,문치르,1,공동인증서 로그인하려니 폐기된 인증서라함. 다시 인증서 받아오려니 이미 유효한 인증...
